### Quick script to get gene or CDS sequences from gff file

In [1]:
import pandas as pd
import pybedtools 
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import collections

In [2]:
in_dir = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
feature = "CDS"

In [3]:
def col_8_id(x):
    import re
    pattern = r'ID=([a-zA-Z0-9_.]*)'
    regex = re.compile(pattern)
    m = regex.search(x)
    match = m.groups()[0].replace('TU', 'model')
    if match.startswith('cds.'):
        match = match[4:]
    if 'exon' in match:
        _list = match.split('.')
        match = '.'.join(_list[:-1])
    return match

In [16]:
anno_gff = [x for x in os.listdir(in_dir) if x.endswith('anno.gff3')]
anno_gff.sort()
genome_fa = [x for x in os.listdir(in_dir) if x.endswith('_ctg.fa')]
genome_fa.sort()
if len(anno_gff) != len(genome_fa):
    print('Check if all anno.gff3 and genome fa files are available. It should be three each!')
print(genome_fa)
print(anno_gff)

['Pst_104E_v12_h_ctg.fa', 'Pst_104E_v12_p_ctg.fa', 'Pst_104E_v12_ph_ctg.fa']
['Pst_104E_v12_h_ctg.anno.gff3', 'Pst_104E_v12_p_ctg.anno.gff3', 'Pst_104E_v12_ph_ctg.anno.gff3']


In [18]:
if feature == 'CDS':
    for i, gff in enumerate(anno_gff):
        print(gff)
        gff_df = pd.read_csv(in_dir+'/'+gff, sep='\t', header=None)
        gff_df_CDS = gff_df[gff_df[2].str.contains(feature)]
        gff_df_CDS[2] = gff_df_CDS[8].apply(col_8_id)
        out_fn = gff.replace('anno', 'anno.'+feature)
        gff_df_CDS.to_csv(in_dir+'/'+out_fn, sep='\t', header=None, index=None)
        #here generate a gff where the name column (==3) contains the ID of the CDS.start_end_strand
        gff_df_CDS[2] = gff_df_CDS[2]+'.'+gff_df_CDS[3].astype(str)+'_'+gff_df_CDS[4].astype(str)+'_'+gff_df_CDS[6].astype(str)+'_'+gff_df_CDS[7].astype(str) 
        out_fn += '.tmp'
        gff_df_CDS.to_csv(in_dir+'/'+out_fn, sep='\t', header=None, index=None)
        out_fn_2 = out_fn.replace('gff3','fa')
        !bedtools getfasta -s -name -fullHeader -fi {in_dir+'/'+genome_fa[i]} -bed {in_dir+'/'+out_fn} -fo {in_dir+'/'+out_fn_2}

Pst_104E_v12_h_ctg.anno.gff3


/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Pst_104E_v12_p_ctg.anno.gff3
Pst_104E_v12_ph_ctg.anno.gff3


In [23]:
#now pull in the tmp file generated by the bedtools
CDS_dict ={}
CDS_sequences = {}
protein_sequences = {}
non_phase_dict = {}
count = 0
not_multiples = []
#generate a dictionary of CDS sequences. Key being the CDS id and value being the dict key=partial_CDS_ID
#value being the sequence objects
for gff_CDS_tmp in [x for x in os.listdir(in_dir) if x.endswith('CDS.fa.tmp')]:
    print("Working on %s." %(gff_CDS_tmp))
    for record in SeqIO.parse(in_dir+'/'+gff_CDS_tmp, 'fasta'):
        _id = '.'.join(record.id.split('.')[:-1])
        if _id not in CDS_dict.keys():
            CDS_dict[_id] = {record.id:record.seq}
        else:
            CDS_dict[_id].update({record.id:record.seq})

    #loop through the dictonary. Make an ordered dict for each CDS dict and concatenate the sequences
    for record_id in CDS_dict.keys():
        #record_id ='evm.model.hcontig_050_003.3'
        tmp_dict = CDS_dict[record_id]
        tmp_start_dict = {}
        for x in tmp_dict.keys():
            #need to get the keys as intergers to sort them numerically
            start = int(x.split('.')[-1].split('_')[0])
            tmp_start_dict.update({start:x})
        tmp_od_dict = collections.OrderedDict(sorted(tmp_start_dict.items(), key=lambda t:t[0]))
        sequence =''
        #check if the sequence is reversed using the strand information
        if set([x[-3] for x in tmp_dict.keys()]) == {'-'}:
            for x in reversed(tmp_od_dict):
                key = tmp_od_dict[x]
                sequence = sequence + tmp_dict[key]
                sub_set = [tmp_od_dict[x] for x in reversed(tmp_od_dict)][0][-1]
                if sub_set != '0':
                    non_phase_dict.update({record_id:sub_set})

        elif set([x[-3] for x in tmp_dict.keys()]) == {'+'}:
            for x in tmp_od_dict.keys():
                key = tmp_od_dict[x]
                sequence = sequence + tmp_dict[key]
                sub_set = sub_set = [tmp_od_dict[x] for x in tmp_od_dict][0][-1]
                if sub_set != '0':
                    non_phase_dict.update({record_id:sub_set})


        else:
            print('Something with the parsing went wrong!')
            print(record_id)
            #print(sequences)

        if len(sequence)%3 != 0:
            #print(record_id)
            not_multiples.append(record_id)
        #print(sequences)
        prot_seq = sequence.translate()
        CDS_record = SeqRecord(sequence, id=record_id, description='')
        protein_record = SeqRecord(prot_seq, id=record_id, description='')
        CDS_sequences.update({record_id : CDS_record})
        protein_sequences.update({record_id :protein_record})

    out_fn_CDS = in_dir+'/'+out_fn_2.replace('.tmp','')
    out_fn_protein = out_fn_CDS.replace("CDS", 'protein2')
    out_hn_CDS = open(out_fn_CDS, 'w')
    SeqIO.write(CDS_sequences.values(), out_hn_CDS, 'fasta')
    out_hn_CDS.close()
    for x in non_phase_dict.keys():
        sub_set = int(non_phase_dict[x])
        sequence = CDS_sequences[x].seq
        prot_seq = sequence[sub_set:].translate()
        protein_record = SeqRecord(prot_seq, id=x, description='')
        protein_sequences.update({x :protein_record})
    out_hn_protein = open(out_fn_protein, 'w')
    SeqIO.write(protein_sequences.values(), out_fn_protein, 'fasta')
    out_hn_protein.close()

Working on Pst_104E_v12_p_ctg.anno.CDS.fa.tmp.


/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/Seq.py:2095: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


Working on Pst_104E_v12_h_ctg.anno.CDS.fa.tmp.
Working on Pst_104E_v12_ph_ctg.anno.CDS.fa.tmp.


In [24]:
count = 0
for x in protein_sequences.keys():
    if protein_sequences[x].seq.count('*') > 1:
        print(protein_sequences[x])
        count += 1
print(count)

0


In [25]:
for x in SeqIO.parse(in_dir+'/Pst_104E_v12_ph_ctg.anno.protein.fa', format='fasta'):
    if x.seq != protein_sequences[x.id].seq:
        print(x.id)

evm.model.hcontig_031_015.8


In [ ]:
len({x for x in non_phase_dict.keys()} -set(not_multiples) )

In [ ]:
for x in non_phase_dict.keys():
    x = 'evm.model.hcontig_031_015.8'
    sub_set = int(non_phase_dict[x])
    sequence = CDS_sequences[x].seq
    protein_seq = sequence[sub_set:].translate()
    print(protein_seq)

In [ ]:
sequence[1:]

In [ ]:
non_phase_dict['evm.model.hcontig_035_013.1']

In [ ]:
[x for x in reversed(tmp_od_dict.keys())][0][-3]

In [ ]:
non_phase_dict['evm.model.hcontig_014_117.28']

In [ ]:
for x in non_phase_dict.keys():
    #sub_set = non_phase_dict['evm.model.hcontig_014_117.28']
    print(CDS_sequences['evm.model.hcontig_014_117.28'].seq.translate())
    print(sub_set)

In [ ]:

tmp_od_dict = collections.OrderedDict(sorted(CDS_dict['evm.model.hcontig_026_011.1'].items()))
sequence =''
non_phase_dict = {}


if set([x[-3] for x in tmp_od_dict.keys()]) == {'-'}:
    sub_set = [x for x in tmp_od_dict.keys()][0][-1]
    if sub_set != '.':
        non_phase_dict.update({'.'.join(x.split('.')[:-1]):sub_set})
    for x in tmp_od_dict.keys():
        print(x)
        sequence = sequence + tmp_od_dict[x]
elif set([x[-3] for x in tmp_od_dict.keys()]) == {'+'}:
        sub_set = [x for x in tmp_od_dict.keys()][0][-1]
        if sub_set != '.':
            non_phase_dict.update({'.'.join(x.split('.')[:-1]):sub_set})
        for x in tmp_od_dict:
            sequence = sequence + tmp_od_dict[x]
else:
    print('Something with the parsing went wrong!')
    print(record_id)
        #print(sequences)
if len(sequence)%3 != 0:

    print("Seqence for %s not multiple of 3" % x)
print(non_phase_dict)

In [ ]:
for x in tmp_od_dict.keys():
        print(x.split('.')[-1].split('_')[0])

In [ ]:
for x in tmp_od_dict.:
    print (x)

In [ ]:
[x[-1] for x in tmp_od_dict.keys()]

In [ ]:
non_phase_dict['evm.model.hcontig_014_117.28']

In [ ]:
[x for x in reversed(tmp_od_dict.keys())]

In [ ]:
reversed(tmp_od_dict.keys())

In [ ]:
CDS_dict['evm.model.hcontig_000_037.3']

In [ ]:
sequence.translate()

In [ ]:
seq =''
for x in CDS_dict['evm.model.hcontig_000_037.3'].keys():
    print(CDS_dict['evm.model.hcontig_000_037.3'][x])
    

In [ ]:
print(seq)

In [ ]:
len(sequence)%3

In [ ]:
len(CDS_sequences)

In [ ]:
asequence = sequence + tmp_od_dict[x]

In [ ]:
tmp_od_dict[x]

In [ ]:
prot_seq = sequence.translate()
prot_string = str(prot_seq)
prot_string

In [ ]:
SeqRecord(Seq("MKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF",\
                     ),\
id="YP_025292.1", name="HokC",\
description="toxic membrane protein")

In [ ]:
SeqRecord(seq = sequence.translate(cds=True), \
                     id = "trans_", \
                     description = "translation of CDS, using default table")

In [ ]:
prot_rec = SeqRecord(seq=prot_seq )
prot_rec.seq

In [ ]:
CDS_sequences[4]

In [12]:
tmp_files = [in_dir+'/'+x for x in os.listdir(in_dir) if x.endswith('tmp') ]

In [13]:
for x in tmp_files:
    os.remove(x)

In [ ]:
prot_seq

In [ ]:
bedtool = pybedtools.BedTool(in_dir+'/Pst_104E_v12_h_ctg.anno.CDS.gff3')

In [ ]:
!bedtools getfasta -s -name -fullHeader -fi {in_dir+'/'+genome_fa[0]} -bed {in_dir+'/'+out_fn} -fo {in_dir+'/'}test.fa

In [ ]:
len(record.seq)

In [ ]:
gff_df_CDS.head(20)

In [ ]:
import collections
#od_CDS_dict = collections.OrderedDict(sorted())
tmp_od_dict = collections.OrderedDict(sorted(CDS_dict['evm.model.hcontig_022_015'].items()))
if set([x[-1] for x in tmp_od_dict.keys()]) == {'-'}:
       print('True')

In [ ]:
tmp_od_dict

In [ ]:
seq = ''
for x in reversed(tmp_od_dict):
    seq += tmp_od_dict[x]

In [ ]:
print(seq.translate())

In [ ]:
from Bio import SeqRecord.SeqRecord

In [ ]:
_record = SeqRecord.SeqRecord(seq, id='evm.model.hcontig_000_003.181', description='')

In [ ]:
out_hn = open(in_dir+'/test2.fa', 'w')
SeqIO.write(_record, out_hn, 'fasta')
out_hn.close()

In [ ]:
_record.seq = seq

In [ ]:
_record.format = 'fasta'
_record.seq = 

In [ ]:
tmp_od_dict

In [ ]:
od_CDS_dict.keys()